In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1. Preparation of Data and Exploratory Data Analysis

In [ ]:
import os
import zipfile
import random
import tensorflow as tf
import shutil
import numpy as np
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from shutil import copyfile
from os import getcwd

In [ ]:
local_zip = '../input/dogs-vs-cats-redux-kernels-edition/train.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/kaggle/working/')
zip_ref.close()

In [ ]:
base_dir = '/kaggle/working/'
train_dir = os.path.join(base_dir, 'train')
train_img_names = os.listdir(train_dir)

In [ ]:
train_img_names[:10]

In [ ]:
print('total training images :', len(train_img_names ))

In [ ]:
categories= list()
for image in train_img_names:
    category = image.split(".")[0]
    if category == "dog":
        categories.append("dog")
    else:
        categories.append("cat")
df= pd.DataFrame({"Image":train_img_names, "Category": categories})

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import random
plt.figure(figsize=(12,10))
sns.countplot(data=df, x="Category",palette="magma")

In [ ]:
sample = random.choice(train_img_names)
plt.imshow(plt.imread(("/kaggle/working/train/"+sample)))

In [ ]:
sample = random.choice(train_img_names)
plt.imshow(plt.imread(("/kaggle/working/train/"+sample)))

In [ ]:
from sklearn.model_selection import train_test_split
train,validation= train_test_split(df, test_size=0.1)
train = train.reset_index(drop=True)
validation = validation.reset_index(drop=True)

In [ ]:
train

In [ ]:
validation

In [ ]:
plt.figure(figsize=(13,10))
sns.countplot(data=train, x="Category",palette="viridis")

In [ ]:
plt.figure(figsize=(13,10))
sns.countplot(data=validation, x="Category",palette="plasma")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )


# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_dataframe(train,
                                                    directory="./train",
                                                    x_col='Image',
                                                    y_col='Category',
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))   

In [ ]:
validation_datagen  = ImageDataGenerator( rescale = 1.0/255.)
validation_generator =  validation_datagen.flow_from_dataframe(validation,
                                                            directory="./train",
                                                              x_col='Image',
                                                             y_col='Category',
                                                              batch_size=20,
                                                              class_mode  = 'binary',
                                                              target_size = (150, 150))

## 2. Building a Small Model from Scratch

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3,activation="relu", input_shape=(150,150,3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2, strides=2))
model.add(Conv2D(filters=64, kernel_size=3,activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2, strides=2))
model.add(Conv2D(filters=128, kernel_size=3, activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2, strides=2))
model.add(Conv2D(filters=256, kernel_size=3, activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2, strides=2))
model.add(Conv2D(filters=256, kernel_size=3, activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2, strides=2))
model.add(Flatten())
model.add(Dense(units=1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation="sigmoid"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
callback=EarlyStopping(monitor="val_loss", patience=2)
callback_lr = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor='val_acc', patience=2, factor=0.5, min_lr=0.00001)

In [ ]:


history=model.fit(train_generator, validation_data=validation_generator, 
                  epochs=8, callbacks=[callback,callback_lr])

## 3. Performance Evaluation of the Training

In [ ]:
pd.DataFrame(model.history.history)

In [ ]:
sns.set_style("darkgrid")
pd.DataFrame(model.history.history).plot(figsize=(15,10))

In [ ]:
acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )

## 4. Preparing Test Data and Submission

In [ ]:
# Extracting the zip file

test_zip = '../input/dogs-vs-cats-redux-kernels-edition/test.zip'
zip_ref = zipfile.ZipFile(test_zip, 'r')
zip_ref.extractall('/kaggle/working')
zip_ref.close()



In [ ]:
test_dir = '/kaggle/working/test/'
test_images = os.listdir(os.path.join(test_dir))
test_images[:10]

In [ ]:
test_df = pd.DataFrame({'Image': test_images})
test_df.head()

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(test_df,
                                                  directory="/kaggle/working/test",
                                                 x_col="Image",
                                                 y_col=None,
                                                  class_mode  = None,
                                                 target_size=(150,150),
                                                shuffle = True,
                                                batch_size=20)

In [ ]:
predictions = model.predict(test_generator,steps = np.ceil(12500/20))
predictions

In [ ]:
test_df["category"]=pd.DataFrame(predictions, columns=["category"])
test_df

In [ ]:
def labelizor(prediction):
    if prediction > 0.5:
        return 1
    else:
        return 0

In [ ]:
test_df["category"] = test_df["category"].apply(labelizor)
test_df

In [ ]:
plt.figure(figsize=(13,10))
sns.countplot(data=test_df, x="category",palette="magma")

In [ ]:
test_df=test_df.reset_index()
test_df

In [ ]:
test_df=test_df.rename(columns={"index": "id"})
test_df

In [ ]:
submission_df=test_df.copy()
submission_df.drop("Image", axis=1, inplace=True)
submission_df

In [ ]:
submission_df.to_csv('submission.csv', index=False)